In [1]:
!wget https://www.dropbox.com/s/pdhwlpi2yeie0ol/movie-dataset.zip

--2022-08-04 23:56:32--  https://www.dropbox.com/s/pdhwlpi2yeie0ol/movie-dataset.zip
Resolving www.dropbox.com (www.dropbox.com)... 162.125.5.18, 2620:100:601d:18::a27d:512
Connecting to www.dropbox.com (www.dropbox.com)|162.125.5.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/pdhwlpi2yeie0ol/movie-dataset.zip [following]
--2022-08-04 23:56:33--  https://www.dropbox.com/s/raw/pdhwlpi2yeie0ol/movie-dataset.zip
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://ucbc9f646c99b6c93812574f8969.dl.dropboxusercontent.com/cd/0/inline/BqZV3Mbn8Ca_3oU5_671onKPYIMGdhfHcemH5r61KJJaixIwFsnvBVzYyXnGIH7XvcrOf_2cZOZYePyyfDTnSIPS_MKGGIcksTxsHDMGY7gos2S_sgeEdWGwrA5BlRraA0U2KZLuTboWi38St5nO08Q87Lm_T1S1NN60y6Kl5ByUxA/file# [following]
--2022-08-04 23:56:33--  https://ucbc9f646c99b6c93812574f8969.dl.dropboxusercontent.com/cd/0/inline/BqZV3Mbn8Ca_3oU5_671onKPYIMGdhfHcemH5r61KJJaixIwFsnvB

In [2]:
!unzip -q 'movie-dataset.zip'

In [3]:
from tensorflow.keras.preprocessing import text_dataset_from_directory
from tensorflow.strings import regex_replace
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
from tensorflow.keras.models import Sequential
from tensorflow.keras import Input
from tensorflow.keras.layers import Dense, LSTM, SimpleRNNCell, Embedding, Dropout, GRU, Bidirectional

In [4]:
def prepareData(dir):
  data = text_dataset_from_directory(dir)
  return data.map(
      lambda text, label: (regex_replace(text, '<br />',' '), label),)

In [5]:
train_data = prepareData('movie-reviews-dataset/train')
test_data = prepareData('movie-reviews-dataset/test')

for text_batch, label_batch in train_data.take(1):
  print(text_batch.numpy()[0])
  print(label_batch.numpy()[0])

Found 25000 files belonging to 2 classes.
Found 25000 files belonging to 2 classes.
b"This episode introduced the Holodeck to the TNG world. The Jarada have to be contacted and a precise greeting must be delivered or it would greatly insult them. A tired Picard decides to take a trip into the Holodeck and a wonderful adventure begins. The settings are superb and almost movie like. Alas, the Jarada probe sent shortly thereafter damages the holodeck and all it's safety devices stop working. Picard and now guests must outwit the mobsters of gangland 40s America and return to the Jarada rendezvous. Picard greets the Jarada correctly and a new day dawns between Humanity and the Jarada. This gem of a first season episode set the holodeck for many interesting and unusual adventures to be had there"
1


In [6]:
model = Sequential()
model.add(Input(shape=(1,),dtype='string'))

In [7]:
max_tokens = 1000
max_len = 100

vectorized_layer = TextVectorization(max_tokens = max_tokens,
                                     output_mode = 'int',
                                     output_sequence_length = max_len,)





In [8]:
train_text = train_data.map(lambda text, label : text)
vectorized_layer.adapt(train_text)

model.add(vectorized_layer)

In [9]:
# creating an embedding layer

model.add(Embedding(max_tokens + 1, 128))
# rnn = RNN(SimpleRNNCell(64), return_sequences = False, return_state = False) # can change it from here
model.add(Bidirectional(GRU(64)))
model.add(Dense(64, activation = 'relu'))
model.add(Dense(1, activation = 'sigmoid'))

In [10]:
model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

In [11]:
model.fit(train_data, epochs = 10)

Epoch 1/10
782/782 [==============================] - 99s 122ms/step - loss: 0.5442 - accuracy: 0.7132
Epoch 2/10
782/782 [==============================] - 94s 121ms/step - loss: 0.4269 - accuracy: 0.8023
Epoch 3/10
782/782 [==============================] - 96s 122ms/step - loss: 0.3893 - accuracy: 0.8229
Epoch 4/10
782/782 [==============================] - 96s 122ms/step - loss: 0.3547 - accuracy: 0.8420
Epoch 5/10
782/782 [==============================] - 97s 124ms/step - loss: 0.3264 - accuracy: 0.8549
Epoch 6/10
782/782 [==============================] - 96s 123ms/step - loss: 0.2976 - accuracy: 0.8677
Epoch 7/10
782/782 [==============================] - 98s 125ms/step - loss: 0.2698 - accuracy: 0.8806
Epoch 8/10
782/782 [==============================] - 97s 124ms/step - loss: 0.2443 - accuracy: 0.8929
Epoch 9/10
782/782 [==============================] - 97s 124ms/step - loss: 0.2142 - accuracy: 0.9071
Epoch 10/10
782/782 [==============================] - 98s 125ms/step - l

In [12]:
model.evaluate(test_data)

782/782 [==============================] - 28s 35ms/step - loss: 0.7817 - accuracy: 0.7658


[0.7817400693893433, 0.7658399939537048]

In [13]:
model.predict(['I loved the movie'])

array([[0.9963293]], dtype=float32)